Cell 1 : Write the Code to Fetch Data

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
import os  # For file paths



# Full list of 64 districts
DISTRICTS = [
    "Bagerhat,Bangladesh", "Bandarban,Bangladesh", "Barguna,Bangladesh", "Barisal,Bangladesh", "Bhola,Bangladesh",
    "Bogra,Bangladesh", "Brahmanbaria,Bangladesh", "Chandpur,Bangladesh", "Chittagong,Bangladesh", "Chuadanga,Bangladesh",
    "Comilla,Bangladesh", "Cox's Bazar,Bangladesh", "Dhaka,Bangladesh", "Dinajpur,Bangladesh", "Faridpur,Bangladesh",
    "Feni,Bangladesh", "Gaibandha,Bangladesh", "Gazipur,Bangladesh", "Gopalganj,Bangladesh", "Habiganj,Bangladesh",
    "Jamalpur,Bangladesh", "Jessore,Bangladesh", "Jhalokati,Bangladesh", "Jhenaidah,Bangladesh", "Joypurhat,Bangladesh",
    "Khagrachhari,Bangladesh", "Khulna,Bangladesh", "Kishoreganj,Bangladesh", "Kurigram,Bangladesh", "Kushtia,Bangladesh",
    "Lakshmipur,Bangladesh", "Lalmonirhat,Bangladesh", "Madaripur,Bangladesh", "Magura,Bangladesh", "Manikganj,Bangladesh",
    "Meherpur,Bangladesh", "Moulvibazar,Bangladesh", "Munshiganj,Bangladesh", "Mymensingh,Bangladesh", "Naogaon,Bangladesh",
    "Narail,Bangladesh", "Narayanganj,Bangladesh", "Narsingdi,Bangladesh", "Natore,Bangladesh", "Nawabganj,Bangladesh",
    "Netrakona,Bangladesh", "Nilphamari,Bangladesh", "Noakhali,Bangladesh", "Pabna,Bangladesh", "Panchagarh,Bangladesh",
    "Patuakhali,Bangladesh", "Pirojpur,Bangladesh", "Rajbari,Bangladesh", "Rajshahi,Bangladesh", "Rangamati,Bangladesh",
    "Rangpur,Bangladesh", "Satkhira,Bangladesh", "Shariatpur,Bangladesh", "Sherpur,Bangladesh", "Sirajganj,Bangladesh",
    "Sunamganj,Bangladesh", "Sylhet,Bangladesh", "Tangail,Bangladesh", "Thakurgaon,Bangladesh"
]  # Exactly 64

# Date range
START_YEAR = 1980
END_YEAR = 2024

# Function to fetch year (same as Cell 2)
def fetch_weather_year(location, year):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}?unitGroup=metric&include=days&key={API_KEY}&contentType=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        days = data.get("days", [])
        if not days:
            print(f"No data for {location} {year}")
        else:
            print(f"Fetched {len(days)} days for {location} {year}")
        return days
    except Exception as e:
        print(f"Error for {location} {year}: {str(e)}")
        return []

Cell 2 : Function to Fetch One Year's Data

In [2]:
# Choose which districts to fetch (e.g., first 5 for testing)
districts_to_fetch = DISTRICTS[:5]  # Change to DISTRICTS for all

for location in districts_to_fetch:
    print(f"\nStarting district: {location}")
    all_data = []  # Reset for each district
    
    for year in range(START_YEAR, END_YEAR + 1):
        year_data = fetch_weather_year(location, year)
        all_data.extend(year_data)
        time.sleep(2)  # Delay
    
    # Process immediately for this district
    extracted_data = []
    for day in all_data:
        date_str = day.get("datetime")
        if date_str:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            if dt > datetime(2024, 12, 31):
                continue
            day_val = dt.day
            month_val = dt.month
            year_val = dt.year
        else:
            day_val = month_val = year_val = None
        
        row = {
            "district": location.split(",")[0],  # Add district column
            "day": day_val,
            "month": month_val,
            "year": year_val,
            "temperature(degree C)": day.get("temp"),
            "feels_like(degree C)": day.get("feelslike"),
            "max_temperature(degree C)": day.get("tempmax"),
            "minimum_temperature(degree C)": day.get("tempmin"),
            "humidity": day.get("humidity"),
            "precipitation": day.get("precip"),
            "windspeed": day.get("windspeed"),
            "atmospheric_pressure": day.get("pressure"),
            "UV": day.get("uvindex"),
            "solar_radiation": day.get("solarradiation"),
            "dew_point": day.get("dew")
        }
        extracted_data.append(row)
    
    # Save per district
    df = pd.DataFrame(extracted_data)
    filename = f"{location.split(',')[0]}_historical_weather_1980_2024.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} records for {location} to {filename}")
    
    # Optional: Combine all so far
    # all_files = [f for f in os.listdir('.') if f.endswith('.csv')]
    # combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
    # combined_df.to_csv("all_bangladesh_weather_1980_2024.csv", index=False)


Starting district: Bagerhat,Bangladesh
Fetched 366 days for Bagerhat,Bangladesh 1980
Fetched 365 days for Bagerhat,Bangladesh 1981
Fetched 365 days for Bagerhat,Bangladesh 1982
Fetched 365 days for Bagerhat,Bangladesh 1983
Fetched 366 days for Bagerhat,Bangladesh 1984
Fetched 365 days for Bagerhat,Bangladesh 1985
Fetched 365 days for Bagerhat,Bangladesh 1986
Fetched 365 days for Bagerhat,Bangladesh 1987
Fetched 366 days for Bagerhat,Bangladesh 1988
Fetched 365 days for Bagerhat,Bangladesh 1989
Fetched 365 days for Bagerhat,Bangladesh 1990
Fetched 365 days for Bagerhat,Bangladesh 1991
Fetched 366 days for Bagerhat,Bangladesh 1992
Fetched 365 days for Bagerhat,Bangladesh 1993
Fetched 365 days for Bagerhat,Bangladesh 1994
Fetched 365 days for Bagerhat,Bangladesh 1995
Fetched 366 days for Bagerhat,Bangladesh 1996
Fetched 365 days for Bagerhat,Bangladesh 1997
Fetched 365 days for Bagerhat,Bangladesh 1998
Fetched 365 days for Bagerhat,Bangladesh 1999
Fetched 366 days for Bagerhat,Bangladesh

cell 2.2 : Loop to Fetch All Years (in Batches)

In [3]:
# Choose which districts to fetch (e.g., first 5 for testing)
districts_to_fetch = DISTRICTS[5:10]  # Bogra to Chuadanga

for location in districts_to_fetch:
    print(f"\nStarting district: {location}")
    all_data = []  # Reset for each district
    
    for year in range(START_YEAR, END_YEAR + 1):
        year_data = fetch_weather_year(location, year)
        all_data.extend(year_data)
        time.sleep(2)  # Delay
    
    # Process immediately for this district
    extracted_data = []
    for day in all_data:
        date_str = day.get("datetime")
        if date_str:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            if dt > datetime(2024, 12, 31):
                continue
            day_val = dt.day
            month_val = dt.month
            year_val = dt.year
        else:
            day_val = month_val = year_val = None
        
        row = {
            "district": location.split(",")[0],  # Add district column
            "day": day_val,
            "month": month_val,
            "year": year_val,
            "temperature(degree C)": day.get("temp"),
            "feels_like(degree C)": day.get("feelslike"),
            "max_temperature(degree C)": day.get("tempmax"),
            "minimum_temperature(degree C)": day.get("tempmin"),
            "humidity": day.get("humidity"),
            "precipitation": day.get("precip"),
            "windspeed": day.get("windspeed"),
            "atmospheric_pressure": day.get("pressure"),
            "UV": day.get("uvindex"),
            "solar_radiation": day.get("solarradiation"),
            "dew_point": day.get("dew")
        }
        extracted_data.append(row)
    
    # Save per district
    df = pd.DataFrame(extracted_data)
    filename = f"{location.split(',')[0]}_historical_weather_1980_2024.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} records for {location} to {filename}")
    
    # Optional: Combine all so far
    # all_files = [f for f in os.listdir('.') if f.endswith('.csv')]
    # combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
    # combined_df.to_csv("all_bangladesh_weather_1980_2024.csv", index=False)


Starting district: Bogra,Bangladesh
Fetched 366 days for Bogra,Bangladesh 1980
Fetched 365 days for Bogra,Bangladesh 1981
Fetched 365 days for Bogra,Bangladesh 1982
Fetched 365 days for Bogra,Bangladesh 1983
Fetched 366 days for Bogra,Bangladesh 1984
Fetched 365 days for Bogra,Bangladesh 1985
Fetched 365 days for Bogra,Bangladesh 1986
Fetched 365 days for Bogra,Bangladesh 1987
Fetched 366 days for Bogra,Bangladesh 1988
Fetched 365 days for Bogra,Bangladesh 1989
Fetched 365 days for Bogra,Bangladesh 1990
Fetched 365 days for Bogra,Bangladesh 1991
Fetched 366 days for Bogra,Bangladesh 1992
Fetched 365 days for Bogra,Bangladesh 1993
Fetched 365 days for Bogra,Bangladesh 1994
Fetched 365 days for Bogra,Bangladesh 1995
Fetched 366 days for Bogra,Bangladesh 1996
Fetched 365 days for Bogra,Bangladesh 1997
Fetched 365 days for Bogra,Bangladesh 1998
Fetched 365 days for Bogra,Bangladesh 1999
Fetched 366 days for Bogra,Bangladesh 2000
Fetched 365 days for Bogra,Bangladesh 2001
Fetched 365 days 

cell 2.3 : Loop to Fetch All Years (in Batches)

In [4]:
# Choose which districts to fetch (e.g., first 5 for testing)
districts_to_fetch = DISTRICTS[10:15]  # Comilla to Faridpur

for location in districts_to_fetch:
    print(f"\nStarting district: {location}")
    all_data = []  # Reset for each district
    
    for year in range(START_YEAR, END_YEAR + 1):
        year_data = fetch_weather_year(location, year)
        all_data.extend(year_data)
        time.sleep(2)  # Delay
    
    # Process immediately for this district
    extracted_data = []
    for day in all_data:
        date_str = day.get("datetime")
        if date_str:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            if dt > datetime(2024, 12, 31):
                continue
            day_val = dt.day
            month_val = dt.month
            year_val = dt.year
        else:
            day_val = month_val = year_val = None
        
        row = {
            "district": location.split(",")[0],  # Add district column
            "day": day_val,
            "month": month_val,
            "year": year_val,
            "temperature(degree C)": day.get("temp"),
            "feels_like(degree C)": day.get("feelslike"),
            "max_temperature(degree C)": day.get("tempmax"),
            "minimum_temperature(degree C)": day.get("tempmin"),
            "humidity": day.get("humidity"),
            "precipitation": day.get("precip"),
            "windspeed": day.get("windspeed"),
            "atmospheric_pressure": day.get("pressure"),
            "UV": day.get("uvindex"),
            "solar_radiation": day.get("solarradiation"),
            "dew_point": day.get("dew")
        }
        extracted_data.append(row)
    
    # Save per district
    df = pd.DataFrame(extracted_data)
    filename = f"{location.split(',')[0]}_historical_weather_1980_2024.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} records for {location} to {filename}")
    
    # Optional: Combine all so far
    # all_files = [f for f in os.listdir('.') if f.endswith('.csv')]
    # combined_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
    # combined_df.to_csv("all_bangladesh_weather_1980_2024.csv", index=False)


Starting district: Comilla,Bangladesh
Fetched 366 days for Comilla,Bangladesh 1980
Fetched 365 days for Comilla,Bangladesh 1981
Fetched 365 days for Comilla,Bangladesh 1982
Fetched 365 days for Comilla,Bangladesh 1983
Fetched 366 days for Comilla,Bangladesh 1984
Fetched 365 days for Comilla,Bangladesh 1985
Fetched 365 days for Comilla,Bangladesh 1986
Fetched 365 days for Comilla,Bangladesh 1987
Fetched 366 days for Comilla,Bangladesh 1988
Fetched 365 days for Comilla,Bangladesh 1989
Fetched 365 days for Comilla,Bangladesh 1990
Fetched 365 days for Comilla,Bangladesh 1991
Fetched 366 days for Comilla,Bangladesh 1992
Fetched 365 days for Comilla,Bangladesh 1993
Fetched 365 days for Comilla,Bangladesh 1994
Fetched 365 days for Comilla,Bangladesh 1995
Fetched 366 days for Comilla,Bangladesh 1996
Fetched 365 days for Comilla,Bangladesh 1997
Fetched 365 days for Comilla,Bangladesh 1998
Fetched 365 days for Comilla,Bangladesh 1999
Fetched 366 days for Comilla,Bangladesh 2000
Fetched 365 days

In [5]:
# Choose the last district to fetch
districts_to_fetch = ["Thakurgaon,Bangladesh"]

for location in districts_to_fetch:
    print(f"\nStarting district: {location}")
    all_data = []  # Reset for each district
    
    for year in range(1980, 2025):  # 2025 is exclusive, stops at 2024
        year_data = fetch_weather_year(location, year)
        all_data.extend(year_data)
        time.sleep(2)  # Delay
    
    # Process immediately for this district
    extracted_data = []
    for day in all_data:
        date_str = day.get("datetime")
        if date_str:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
            if dt > datetime(2024, 12, 31):
                continue
            day_val = dt.day
            month_val = dt.month
            year_val = dt.year
        else:
            day_val = month_val = year_val = None
        
        row = {
            "district": location.split(",")[0],
            "day": day_val,
            "month": month_val,
            "year": year_val,
            "temperature(degree C)": day.get("temp"),
            "feels_like(degree C)": day.get("feelslike"),
            "max_temperature(degree C)": day.get("tempmax"),
            "minimum_temperature(degree C)": day.get("tempmin"),
            "humidity": day.get("humidity"),
            "precipitation": day.get("precip"),
            "windspeed": day.get("windspeed"),
            "atmospheric_pressure": day.get("pressure"),
            "UV": day.get("uvindex"),
            "solar_radiation": day.get("solarradiation"),
            "dew_point": day.get("dew")
        }
        extracted_data.append(row)
    
    # Save per district
    df = pd.DataFrame(extracted_data)
    filename = f"{location.split(',')[0]}_historical_weather_1980_2024.csv"
    if os.path.exists(filename):
        print(f"Skipping {location}, file exists")
        continue
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} records for {location} to {filename}")


Starting district: Thakurgaon,Bangladesh
Fetched 366 days for Thakurgaon,Bangladesh 1980
Fetched 365 days for Thakurgaon,Bangladesh 1981
Fetched 365 days for Thakurgaon,Bangladesh 1982
Fetched 365 days for Thakurgaon,Bangladesh 1983
Fetched 366 days for Thakurgaon,Bangladesh 1984
Fetched 365 days for Thakurgaon,Bangladesh 1985
Fetched 365 days for Thakurgaon,Bangladesh 1986
Fetched 365 days for Thakurgaon,Bangladesh 1987
Fetched 366 days for Thakurgaon,Bangladesh 1988
Fetched 365 days for Thakurgaon,Bangladesh 1989
Fetched 365 days for Thakurgaon,Bangladesh 1990
Fetched 365 days for Thakurgaon,Bangladesh 1991
Fetched 366 days for Thakurgaon,Bangladesh 1992
Fetched 365 days for Thakurgaon,Bangladesh 1993
Fetched 365 days for Thakurgaon,Bangladesh 1994
Fetched 365 days for Thakurgaon,Bangladesh 1995
Fetched 366 days for Thakurgaon,Bangladesh 1996
Fetched 365 days for Thakurgaon,Bangladesh 1997
Fetched 365 days for Thakurgaon,Bangladesh 1998
Fetched 365 days for Thakurgaon,Bangladesh 199